In [2]:
#In this part, we realized MESMO algorithm by modifying open-source code: https://github.com/belakaria/MESMO 
# We set parameters:
# initial_points = 20  Initial points
# input_dim=2,  input dim
# function_dim=2, output dim
# total_iterations=100, MCMC numbers
# paths='.',  paths to save the results
# num_trial=0, to get a stable result, run num_trial times and compute the mean
# n_sample = 5,  another MCMC numbers used in MESMO algorithm


In [3]:
#import multi-objective classes
%run MultiObjectives_version2.ipynb

#import other packages
import numpy as np
from scipy.stats import norm
from sklearn.kernel_approximation import RBFSampler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel,Matern
import os
from scipy.optimize import minimize as scipyminimize
from platypus import NSGAII, Problem, Real
import sobol_seq
from pygmo import hypervolume

In [4]:
# Define GP model 

class GaussianProcess:
    def __init__(self, dim):
        self.dim = dim
        self.kernel =  RBF(length_scale=1, length_scale_bounds=(1e-3, 1e2))
        self.beta=1e6
        self.xValues = []
        self.yValues = []
        self.yValuesNorm=[]
        self.model = GaussianProcessRegressor(kernel=self.kernel,n_restarts_optimizer=5)
        
    def fitNormal(self):
        y_mean = np.mean(self.yValues)
        y_std = self.getstd()
        self.yValuesNorm= (self.yValues - y_mean)/y_std
        self.model.fit(self.xValues, self.yValuesNorm)
        
    def fitModel(self):
        self.model.fit(self.xValues, self.yValues)

    def addSample(self, x, y):
        self.xValues.append(x)
        self.yValues.append(y)

    def getPrediction(self, x):
        mean, std = self.model.predict(x.reshape(1,-1),return_std=True)
        if std[0]==0:
            std[0]=np.sqrt(1e-5)*self.getstd()
        return mean, std

    def getmean(self):
        return np.mean(self.yValues)

    def getstd(self):
        y_std=np.std(self.yValues)
        if y_std==0:
            y_std=1
        return y_std

In [5]:
class MaxvalueEntropySearch(object):
    def __init__(self, GPmodel):
        self.GPmodel = GPmodel
        self.y_max = max(GPmodel.yValues)
        self.d = GPmodel.dim


    def Sampling_RFM(self):
        self.rbf_features = RBFSampler(gamma=1/(2*self.GPmodel.kernel.length_scale**2), n_components=1000, random_state=1) #length_scale=1
        X_train_features = self.rbf_features.fit_transform(np.asarray(self.GPmodel.xValues))

        A_inv = np.linalg.inv((X_train_features.T).dot(X_train_features) + np.eye(self.rbf_features.n_components)/self.GPmodel.beta)
        self.weights_mu = A_inv.dot(X_train_features.T).dot(self.GPmodel.yValues)
        weights_gamma = A_inv / self.GPmodel.beta
        self.L = np.linalg.cholesky(weights_gamma)

    def weigh_sampling(self):
        random_normal_sample = np.random.normal(0, 1, np.size(self.weights_mu))
        self.sampled_weights = np.c_[self.weights_mu] + self.L.dot(np.c_[random_normal_sample])
    def f_regression(self,x):

        X_features = self.rbf_features.fit_transform(x.reshape(1,len(x)))
        return -(X_features.dot(self.sampled_weights)) 

    def single_acq(self, x,maximum):
        mean, std = self.GPmodel.getPrediction(x)
        mean=mean[0]
        std=std[0]
        if maximum < max(self.GPmodel.yValues)+5/self.GPmodel.beta:
            maximum=max(self.GPmodel.yValues)+5/self.GPmodel.beta

        normalized_max = (maximum - mean) / std
        pdf = norm.pdf(normalized_max)
        cdf = norm.cdf(normalized_max)
        if (cdf==0):
            cdf=1e-30
        return   -(normalized_max * pdf) / (2*cdf) + np.log(cdf)




In [8]:
def MESMO_multiobj(functions, input_dim=2, function_dim=2, total_iterations=100, paths='.', num_trial=0, n_sample = 5, initial_points = 20,ref_point=500):

    import warnings
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    ## Initial samples
    dim = function_dim 
    d = input_dim  
    intial_number = initial_points
    sample_number = n_sample
    referencePoint = functions.get_ref_point(ref_point)
    lb = functions.lowerbounds
    ub = functions.upperbounds
    Fun_bounds = [[0, 0]] * d  
    for i in range(d):
        Fun_bounds[i] = [lb[i], ub[i]]
    
    # Initial GP model
    seed = 0
    np.random.seed(seed)
    grid_scale = sobol_seq.i4_sobol_generate(d, 1000, np.random.randint(0, 100))
    grid = grid_scale
    for i in range(d):
        grid[:, i] = lb[i] + grid_scale[:, i] * (ub[i] - lb[i])
    design_index = np.random.randint(0, grid.shape[0])

    GPs = []
    Multiplemes = []
    for i in range(dim):  
        GPs.append(GaussianProcess(d))
    for k in range(intial_number):
        exist = True
        while exist:
            design_index = np.random.randint(0, grid.shape[0])
            x_rand = list(grid[design_index: (design_index + 1), :][0])
            if (any((x_rand == x).all() for x in GPs[0].xValues)) == False:
                exist = False
        for i in range(dim):
            GPs[i].addSample(np.asarray(x_rand), functions.MinMax_evaluate(x_rand)[i])
    for i in range(dim):  
        GPs[i].fitModel()
        Multiplemes.append(MaxvalueEntropySearch(GPs[i]))

    # write results into file
    if num_trial == 0 and os.path.exists(os.path.join(paths, functions.name + '_MESMO_front.txt')):
        os.remove(os.path.join(paths, functions.name + '_MESMO_front.txt'))
    input_output = open(os.path.join(paths, functions.name + '_MESMO_front.txt'), "a")
    input_output.write("num_trial: " + str(num_trial) + '\n')

    for j in range(len(GPs[0].yValues)):
        input_output_line = ''
        for i in range(dim):
            input_output_line = input_output_line + str(GPs[i].yValues[j]) + ' '
        input_output_line.strip()
        input_output_line = input_output_line + '\n'
        input_output.write(input_output_line)
    input_output.close()

    if num_trial == 0 and os.path.exists(os.path.join(paths, functions.name + '_MESMO_hypervolume.txt')):
        os.remove(os.path.join(paths, functions.name + '_MESMO_hypervolume.txt'))
    current_hypervolume = open(os.path.join(paths, functions.name + '_MESMO_hypervolume.txt'), "a")
    current_hypervolume.write("num_trial: " + str(num_trial) + '\n')
    simple_pareto_front_evaluations = list(zip(*[GPs[i].yValues for i in range(dim)]))
    print("hypervolume ", hypervolume(-1 * (np.asarray(simple_pareto_front_evaluations))).compute(referencePoint))
    current_hypervolume.write('%f \n' % hypervolume(-1 * (np.asarray(simple_pareto_front_evaluations))).compute(referencePoint))
    current_hypervolume.close()

    ############ MESMO 
    for l in range(total_iterations):
        # sample fi
        for i in range(dim):
            Multiplemes[i] = MaxvalueEntropySearch(GPs[i])
            Multiplemes[i].Sampling_RFM()
        max_samples = []

        #MCMC
        for j in range(sample_number):
            for i in range(dim):
                Multiplemes[i].weigh_sampling()
            cheap_pareto_front = []

            def CMO(xi):
                xi = np.asarray(xi)
                y = [Multiplemes[i].f_regression(xi)[0][0] for i in range(len(GPs))]
                return y

            problem = Problem(d, dim)
            problem.types[:] = [Real(lb[i], ub[i]) for i in range(d)]
            problem.function = CMO
            algorithm = NSGAII(problem)
            algorithm.run(1500)
            cheap_pareto_front = [list(solution.objectives) for solution in algorithm.result]
            maxoffunctions = [-1 * min(f) for f in list(zip(*cheap_pareto_front))]
            max_samples.append(maxoffunctions)
            
        # max acquisition function
        def mesmo_acq(x):
            multi_obj_acq_total = 0
            for j in range(sample_number):
                multi_obj_acq_sample = 0
                for i in range(dim):
                    multi_obj_acq_sample = multi_obj_acq_sample + Multiplemes[i].single_acq(x, max_samples[j][i])
                multi_obj_acq_total = multi_obj_acq_total + multi_obj_acq_sample
            return (multi_obj_acq_total / sample_number)

        x_tries_scale = np.random.uniform(low=0, high=1, size=(1000, d))
        x_tries = x_tries_scale
        for i in range(d):
            x_tries[:, i] = lb[i] + x_tries_scale[:, i] * (ub[i] - lb[i])

        y_tries = [mesmo_acq(x) for x in x_tries]
        sorted_indecies = np.argsort(y_tries)
        i = 0
        x_best = x_tries[sorted_indecies[i]]
        while (any((x_best == x).all() for x in GPs[0].xValues)):
            i = i + 1
            x_best = x_tries[sorted_indecies[i]]
        y_best = y_tries[sorted_indecies[i]]

        x_seed_scale = np.random.uniform(low=0, high=1, size=(1000, d))
        #       x_seed_scale = sobol_seq.i4_sobol_generate(d,1000,np.random.randint(0,100))
        x_seed = x_tries_scale
        for i in range(d):
            x_seed[:, i] = lb[i] + x_seed_scale[:, i] * (ub[i] - lb[i])
        x_seed = list(x_seed)

        for x_try in x_seed:
            result = scipyminimize(mesmo_acq, x0=np.asarray(x_try).reshape(1, -1), method='L-BFGS-B', bounds=Fun_bounds)
            if not result.success:
                continue
            if ((result.fun <= y_best) and (not (result.x in np.asarray(GPs[0].xValues)))):
                x_best = result.x
                y_best = result.fun

        # Update
        for i in range(dim):
            GPs[i].addSample(x_best, functions.MinMax_evaluate(list(x_best))[i])
            GPs[i].fitModel()

        # write results
        # samples
        input_output = open(os.path.join(paths, functions.name + '_MESMO_front.txt'), "a")
        input_output_line = ''
        for i in range(dim):
            input_output_line = input_output_line + str(GPs[i].yValues[-1]) + ' '
        input_output_line.strip()
        input_output_line = input_output_line + '\n'
        input_output.write(input_output_line)
        input_output.close()
        
        # hypervolumes
        current_hypervolume = open(os.path.join(paths, functions.name + '_MESMO_hypervolume.txt'), "a")
        simple_pareto_front_evaluations = list(zip(*[GPs[i].yValues for i in range(dim)]))
        print("hypervolume ", hypervolume(-1 * (np.asarray(simple_pareto_front_evaluations))).compute(referencePoint))
        current_hypervolume.write('%f \n' % hypervolume(-1 * (np.asarray(simple_pareto_front_evaluations))).compute(referencePoint))
        current_hypervolume.close()

